In [1]:
import scanpy as sc

In [2]:
microglia_adata = sc.read_h5ad('rds/microglia_adata.h5ad')

In [3]:
# 将microglia_adata 's obs 中的 group 列转换为分类变量, meanwhile make WT before AD
microglia_adata.obs['group'] = microglia_adata.obs['group'].astype('category')
microglia_adata.obs['group'] = microglia_adata.obs['group'].cat.reorder_categories(['WT', 'AD'], ordered=True)


In [4]:
microglia_adata.obs['group'].value_counts

<bound method IndexOpsMixin.value_counts of AAAGAACTCTCAATCT-1-6m_AD_rep1    AD
AAAGTCCAGGATACGC-1-6m_AD_rep1    AD
AAAGTCCAGTAGACAT-1-6m_AD_rep1    AD
AAAGTCCGTTAGCGGA-1-6m_AD_rep1    AD
AACACACAGACCGCCT-1-6m_AD_rep1    AD
                                 ..
TTGGGTACATTAGGCT-1-6m_WT_rep3    WT
TTGTTGTGTCGAGATG-1-6m_WT_rep3    WT
TTTATGCGTGGATTTC-1-6m_WT_rep3    WT
TTTGATCGTACTTCCC-1-6m_WT_rep3    WT
TTTGTTGAGAATCGAT-1-6m_WT_rep3    WT
Name: group, Length: 853, dtype: category
Categories (2, object): ['WT' < 'AD']>

In [8]:
# 对group中的WT和AD，分别计算吞噬评分，然后比较高低和p值。小鼠基因
gene_list = ['C1qa', 'C1qb', 'C1qc', 'C3', 'C3ar1', 'Csf1r', 'Trem2']
gene_list = [gene for gene in gene_list if gene in microglia_adata.var_names]
sc.tl.score_genes(microglia_adata, gene_list=gene_list, score_name='phagocytosis_score')
# 计算group中WT和AD的phagocytosis_score的均值和标准差
phagocytosis_score_summary = microglia_adata.obs.groupby('group')['phagocytosis_score'].agg(['mean', 'std']).reset_index()
# 计算group中WT和AD的phagocytosis_score的p值，使用独立样本t检验
import scipy.stats as stats
scores_WT = microglia_adata.obs.loc[microglia_adata.obs['group'] == 'WT', 'phagocytosis_score']
scores_AD = microglia_adata.obs.loc[microglia_adata.obs['group'] == 'AD', 'phagocytosis_score']
t_stat, phagocytosis_p_value = stats.ttest_ind(scores_WT, scores_AD, nan_policy='omit')
# 打印结果
print("Phagocytosis Score Summary:")
print(phagocytosis_score_summary)
# 打印p值
print("Phagocytosis Score p-value:")
print(phagocytosis_p_value)



Phagocytosis Score Summary:
  group      mean       std
0    WT  0.787644  2.310813
1    AD  1.059726  2.335926
Phagocytosis Score p-value:
0.08773771543425204


/var/folders/00/qm2jk48x1pqbpczsl64nsn1m0000gp/T/ipykernel_3092/2157221643.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  phagocytosis_score_summary = microglia_adata.obs.groupby('group')['phagocytosis_score'].agg(['mean', 'std']).reset_index()


In [10]:
import os
# 保存结果
results_dir = 'results'
os.makedirs(results_dir, exist_ok=True)
phagocytosis_score_summary.to_csv(os.path.join(results_dir, 'phagocytosis_score_summary.csv'), index=False)